In [2]:
import pandas as pd
df = pd.read_csv("merged.csv", low_memory=False)
important_features = [
    "Bwd Packet Length Std", "Flow Bytes/s", "Total Length of Fwd Packets", "Fwd Packet Length Std",
    "Flow IAT Std", "Flow IAT Min", "Fwd IAT Total", "Flow Duration", "Bwd Packet Length Max",
    "Flow IAT Max", "Flow IAT Mean", "Total Length of Bwd Packets", "Fwd Packet Length Min",
    "Bwd Packet Length Mean", "Flow Packets/s", "Fwd Packet Length Mean", "Total Backward Packets",
    "Fwd Packet Length Max", "Total Fwd Packets", "Bwd Packet Length Min", "Label"
]

df_filtered = df[important_features]
df_filtered.to_csv("mergedTop20.csv", index=False)

In [4]:
import os
import time
import joblib
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report

import pennylane as qml
from tqdm import tqdm

# Start timing
start_time = time.time()

# Load dataset
df = pd.read_csv("mergedTop20.csv", low_memory=False)
cols = [
    "Bwd Packet Length Std", "Flow Bytes/s", "Total Length of Fwd Packets", "Fwd Packet Length Std",
    "Flow IAT Std", "Flow IAT Min", "Fwd IAT Total", "Flow Duration", "Bwd Packet Length Max",
    "Flow IAT Max", "Flow IAT Mean", "Total Length of Bwd Packets", "Fwd Packet Length Min",
    "Bwd Packet Length Mean", "Flow Packets/s", "Fwd Packet Length Mean", "Total Backward Packets",
    "Fwd Packet Length Max", "Total Fwd Packets", "Bwd Packet Length Min", "Label"
]
df = df[cols].fillna(0)
df["Label"] = df["Label"].apply(lambda x: 1 if x == "BENIGN" else 0)

# Feature Selection
X = df.drop("Label", axis=1)
y = df["Label"]
top_indices = np.argsort(RandomForestClassifier().fit(X, y).feature_importances_)[::-1][:5]
X_top = X.iloc[:, top_indices]

# Balancing
benign = X_top[y == 1].sample(n=500, random_state=42)
attack = X_top[y == 0].sample(n=500, random_state=42)
X_bal = pd.concat([benign, attack]).reset_index(drop=True)
y_bal = pd.Series([1]*500 + [0]*500)

# Scaling & PCA
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_bal)

pca = PCA(n_components=3)
X_pca = pca.fit_transform(X_scaled)

X_train, X_test, y_train, y_test = train_test_split(X_pca, y_bal, test_size=0.2, stratify=y_bal, random_state=42)

# Quantum kernel definition
n_qubits = X_train.shape[1]
dev = qml.device("default.qubit", wires=n_qubits)

def feature_map(x):
    for i in range(n_qubits):
        qml.Hadamard(wires=i)
        qml.RZ(x[i], wires=i)
    for i in range(n_qubits - 1):
        qml.CNOT(wires=[i, i + 1])
        qml.RZ((x[i] + x[i + 1]) / 2, wires=i + 1)
        qml.CNOT(wires=[i, i + 1])

@qml.qnode(dev)
def kernel_circuit(x1, x2):
    feature_map(x1)
    qml.adjoint(feature_map)(x2)
    return qml.probs(wires=range(n_qubits))

def fidelity(x1, x2):
    return kernel_circuit(x1, x2)[0]

def create_kernel_matrix(X1, X2):
    kernel = np.zeros((len(X1), len(X2)))
    for i in tqdm(range(len(X1)), desc="Building Kernel Rows"):
        for j in range(len(X2)):
            kernel[i, j] = fidelity(X1[i], X2[j])
    return kernel

# Compute kernel matrices
print("Generating training kernel...")
kernel_train = create_kernel_matrix(X_train, X_train)

print("Generating test kernel...")
kernel_test = create_kernel_matrix(X_test, X_train)

# Train SVM
clf = SVC(kernel="precomputed")
clf.fit(kernel_train, y_train)

# Predict and evaluate
y_pred_train = clf.predict(kernel_train)
y_pred_test = clf.predict(kernel_test)

print("\nTraining Report:")
print(classification_report(y_train, y_pred_train))

print("\nTest Report:")
print(classification_report(y_test, y_pred_test))

# Save results
os.makedirs("Pennylane_Result", exist_ok=True)
joblib.dump(clf, "Pennylane_Result/pennylane_qsvc.pkl", compress=3)
joblib.dump(scaler, "Pennylane_Result/pennylane_scaler.pkl", compress=3)
joblib.dump(pca, "Pennylane_Result/pennylane_pca.pkl", compress=3)
joblib.dump(cols[:-1], "Pennylane_Result/pennylane_features_used.pkl", compress=3)

print("Time taken:", round(time.time() - start_time, 2), "seconds")

Generating training kernel...


Building Kernel Rows: 100%|██████████| 800/800 [1:01:49<00:00,  4.64s/it]


Generating test kernel...


Building Kernel Rows: 100%|██████████| 200/200 [11:53<00:00,  3.57s/it]


Training Report:
              precision    recall  f1-score   support

           0       0.93      0.36      0.52       400
           1       0.60      0.97      0.75       400

    accuracy                           0.67       800
   macro avg       0.77      0.67      0.64       800
weighted avg       0.77      0.67      0.64       800


Test Report:
              precision    recall  f1-score   support

           0       1.00      0.45      0.62       100
           1       0.65      1.00      0.78       100

    accuracy                           0.72       200
   macro avg       0.82      0.72      0.70       200
weighted avg       0.82      0.72      0.70       200

Time taken: 5307.85 seconds
